In [2]:
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import qiskit
from qiskit import QuantumCircuit #Aer, IBMQ,
from qiskit.visualization import plot_histogram
from qiskit import transpile
#from qiskit.providers import fake_provider
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime.fake_provider import FakeManilaV2, FakeTorino, FakeBrisbane
from qiskit_ibm_runtime.fake_provider import FakeFez, FakeMarrakesh
#fez, marrakesh are both Heron2
from qiskit.providers.basic_provider import BasicSimulator
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager


In [3]:
import data_extract_funcs

In [5]:
hdwr_8q_tor1 = pd.read_csv("../Hardware_results/8q/8q_torino1.csv")

In [6]:
#hdwr_8q_tor1.replace('',np.nan)
#hdwr_8q_tor1.fillna(0, inplace=True)
hdwr_8q_tor1.head()

,00000000,00000001,00000010,00000011,00000100,00000101,00000110,00000111,00001000,00001001,...,11110110,11110111,11111000,11111001,11111010,11111011,11111100,11111101,11111110,11111111
0,2987,328,201,46,30,9,17,55,36,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,6
1,2946,323,208,52,31,4,14,58,29,9.0,...,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,3.0,8
2,2986,281,193,44,32,11,21,49,46,6.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,2.0,11
3,2977,301,190,51,29,5,13,57,33,6.0,...,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN,NaN,10
4,2950,336,215,44,33,8,15,53,18,1.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,16


## Measuring sparsity of data

In [7]:
non_null_vals = hdwr_8q_tor1.count()
print(non_null_vals)

00000000    50
00000001    50
00000010    50
00000011    50
00000100    50
            ..
11111011    28
11111100     4
11111101    12
11111110    38
11111111    50
Length: 256, dtype: int64


In [4]:
def get_percentage_non_zero(df):
    df.replace(0,'')
    percentage_non_zero = sum(df.count())/(df.size)
    return percentage_non_zero

In [ ]:
percentage_non_zero_8q = sum(non_null_vals)/(hdwr_8q_tor1.size)
print(percentage_non_zero_8q)

0.256875


In [11]:
hdwr_4q_tor1 =  pd.read_csv("../Hardware_results/4q/4q_torino1.csv")
percentage_non_zero_4q = sum(hdwr_4q_tor1.count())/(hdwr_4q_tor1.size)
print(percentage_non_zero_4q)

0.8475877192982456


checking the percentage_non_zero for all data

## Meta_data dataframe

In [5]:
dir_Hardware = ["../Hardware_results/4q/","../Hardware_results/8q/"]
dir_Sims = ["../Simulated_results/4q/","../Simulated_results/8q/","../Simulated_results/16q/"]
dir_Refr_Sims = ["../Refreshed_Simulated_results/4q/","../Refreshed_Simulated_results/8q/","../Refreshed_Simulated_results/16q/"]

In [37]:
df_meta_data = pd.DataFrame(columns = ['nr_qubits','backend','sim','circuit_type','file_path'])
df_meta_data.head()

,nr_qubits,backend,sim,circuit_type,file_path


1. make file names
2. extract nr qubits etc from file names
3. add directory to file names
4. load as df, get percentage non_null

In [36]:
def extract_cols_from_filename(file_name_str, dir_):
    nr_qubits = file_name_str[0:2]
    double_digits = (nr_qubits[-1] != 'q')
    
    if  (not double_digits):
        nr_qubits = nr_qubits[0]

    backend_name_start = len(nr_qubits)+2
    circuit_type = file_name_str[-5]
    is_sim = False
     #might need to 7 change if 'ibm_..." is included in file name"
    if file_name_str.find("fake")!=-1:
        is_sim = True
    if is_sim:
        backend_name_start = backend_name_start +5
    backend_name = file_name_str[backend_name_start:-5]

    file_path = dir_+file_name_str
    
    return [nr_qubits,backend_name,is_sim,circuit_type,file_path]
    

In [ ]:
# test_row =extract_cols_from_filename("16q_torino1.csv","../Hardware/8q/")
# print(test_row)

['16', 'torino', False, '1', '../Hardware/8q/16q_torino1.csv']


In [ ]:
# df_meta_data.loc[len(df_meta_data)] = test_row
# df_meta_data.head()

,nr_qubits,backend,sim,circuit_type,file_path
0,8,torino,False,1,../Hardware/8q/8q_torino1.csv


In [8]:
def add_file_to_meta_df(meta_df,file_name,dir_):
    row = extract_cols_from_filename(file_name,dir_)
    meta_df.loc[len(meta_df)] = row

In [9]:
def add_qfolder_to_df(df,folder_dir,backends_,nr_qubits):
    for backend in backends_:
        backend_name = backend.backend_name
        file_names = data_extract_funcs.make_file_names(backend_name,nr_qubits)
    for file_name in file_names:
        add_file_to_meta_df(df,file_name,folder_dir)

In [10]:
service = QiskitRuntimeService()
fake_backends = [FakeTorino(), FakeFez(), FakeMarrakesh(),FakeBrisbane()]
hardware_backends = [service.backend('ibm_brisbane'), service.backend('ibm_torino')]

In [22]:
add_qfolder_to_df(df_meta_data,"../Simulated_results/4q/",fake_backends,4)
df_meta_data.head()

,nr_qubits,backend,sim,circuit_type,file_path
0,4,brisbane,True,1,../Simulated_results/4q/4q_fake_brisbane1.csv
1,4,brisbane,True,2,../Simulated_results/4q/4q_fake_brisbane2.csv
2,4,brisbane,True,3,../Simulated_results/4q/4q_fake_brisbane3.csv


In [38]:
qs = [4,8,16,32]
for i in range(len(dir_Sims)):
    folder = dir_Sims[i]
    nr_qubits = qs[i]
    #nr_qubits = int(folder[-3])
    add_qfolder_to_df(df_meta_data,folder,fake_backends,nr_qubits)

In [39]:
df_meta_data.head(10)

,nr_qubits,backend,sim,circuit_type,file_path
0,4,brisbane,True,1,../Simulated_results/4q/4q_fake_brisbane1.csv
1,4,brisbane,True,2,../Simulated_results/4q/4q_fake_brisbane2.csv
2,4,brisbane,True,3,../Simulated_results/4q/4q_fake_brisbane3.csv
3,8,brisbane,True,1,../Simulated_results/8q/8q_fake_brisbane1.csv
4,8,brisbane,True,2,../Simulated_results/8q/8q_fake_brisbane2.csv
5,8,brisbane,True,3,../Simulated_results/8q/8q_fake_brisbane3.csv
6,16,brisbane,True,1,../Simulated_results/16q/16q_fake_brisbane1.csv
7,16,brisbane,True,2,../Simulated_results/16q/16q_fake_brisbane2.csv
8,16,brisbane,True,3,../Simulated_results/16q/16q_fake_brisbane3.csv


## Testing different methods for dealing with sparsity
### Removing null columns

### PCA

In [ ]:
#https://note.nkmk.me/en/python-numpy-zeros-ones-full/